# Nanopolish program futtatása.

### How to align events to a reference genome?

The eventalign module in nanopolish is used to align events or “squiggles” to a reference genome. We (the developers of nanopolish) use this feature extensively when we want to see what the low-level signal information looks like. It helps us model the signal and discover differences in current that might hint at base modifications. Here we provide a step-by-step tutorial to help you get started with the nanopolish eventalign module.

gzip formátumba tömörítés

In [25]:
%%bash

path=/media/storage_3/mb/nanopore_data/analysis/r9.4.1/20200914_1354_6B_PAF27096_e7c9eae6/guppy_v4.0.11_r9.4.1_hac_prom/align_unfiltered/chr22

compress_fast5 -i $path/fast5/ -s $path/cmprs_fast5 --compression gzip --recursive 

| 8 of 8|###################################################|100% Time: 0:22:20


In [16]:
%%bash 

fastq=/media/storage_3/mb/nanopore_data/analysis/r9.4.1/20200914_1354_6B_PAF27096_e7c9eae6/guppy_v4.0.11_r9.4.1_hac_prom/align_unfiltered/chr22/guppy/
cd $fastq

cat pass/* > pass_cat/chr22.fastq

### Align reads with minimap2

Index the reads  # for chr22 it takes 3min

In [26]:
%%bash

nanopolish=/home/menkobalazs/nanopolish/nanopolish

fast5=/media/storage_3/mb/nanopore_data/analysis/r9.4.1/20200914_1354_6B_PAF27096_e7c9eae6/guppy_v4.0.11_r9.4.1_hac_prom/align_unfiltered/chr22/cmprs_fast5
fastq=/media/storage_3/mb/nanopore_data/analysis/r9.4.1/20200914_1354_6B_PAF27096_e7c9eae6/guppy_v4.0.11_r9.4.1_hac_prom/align_unfiltered/chr22/guppy/pass_cat/

cd $fastq

$nanopolish index -d $fast5 chr22.fastq

[readdb] indexing /media/storage_3/mb/nanopore_data/analysis/r9.4.1/20200914_1354_6B_PAF27096_e7c9eae6/guppy_v4.0.11_r9.4.1_hac_prom/align_unfiltered/chr22/cmprs_fast5
[readdb] num reads: 56629, num reads with path to fast5: 56629


Next cell's run takes 22min 3sec

In [27]:
%%bash

minimap2=/home/menkobalazs/nanopolish/minimap2/minimap2
samtools=/home/menkobalazs/nanopolish/samtools/bin/samtools

refdir=/media/storage_3/mb/nanopore_data/analysis/ref_sequences/
fastq=/media/storage_3/mb/nanopore_data/analysis/r9.4.1/20200914_1354_6B_PAF27096_e7c9eae6/guppy_v4.0.11_r9.4.1_hac_prom/align_unfiltered/chr22/guppy/pass_cat
dstdir=/media/storage_3/mb/nanopore_data/analysis/r9.4.1/20200914_1354_6B_PAF27096_e7c9eae6/guppy_v4.0.11_r9.4.1_hac_prom/align_unfiltered/chr22/nanopolish/chr22/

mkdir -p $dstdir

$minimap2 -ax map-ont -t 8 $refdir/sequence_chr22.fasta $fastq/chr22.fastq | $samtools sort -o $dstdir/chr22.sorted.bam -T $dstdir/chr22.tmp
$samtools index $dstdir/chr22.sorted.bam

[M::mm_idx_gen::1.263*0.93] collected minimizers
[M::mm_idx_gen::1.543*1.14] sorted minimizers
[M::main::1.543*1.14] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::1.618*1.14] mid_occ = 238
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 1
[M::mm_idx_stat::1.665*1.13] distinct minimizers: 5012420 (87.89% are singletons); average occurrences: 1.472; average spacing: 6.888; total length: 50818468
[M::worker_pipeline::438.285*3.46] mapped 19520 sequences
[M::worker_pipeline::821.762*3.47] mapped 19749 sequences
[M::worker_pipeline::1124.208*3.37] mapped 15639 sequences
[M::main] Version: 2.26-r1175
[M::main] CMD: /home/menkobalazs/nanopolish/minimap2/minimap2 -ax map-ont -t 8 /media/storage_3/mb/nanopore_data/analysis/ref_sequences//sequence_chr22.fasta /media/storage_3/mb/nanopore_data/analysis/r9.4.1/20200914_1354_6B_PAF27096_e7c9eae6/guppy_v4.0.11_r9.4.1_hac_prom/align_unfiltered/chr22/guppy/pass_cat/chr22.fastq
[M::main] Real time: 1124.337 sec; CPU: 

In [28]:
%%bash

samtools=/home/menkobalazs/nanopolish/samtools/bin/samtools

dstdir=/media/storage_3/mb/nanopore_data/analysis/r9.4.1/20200914_1354_6B_PAF27096_e7c9eae6/guppy_v4.0.11_r9.4.1_hac_prom/align_unfiltered/chr22/nanopolish/chr22

$samtools quickcheck $dstdir/chr22.sorted.bam

## Align the events to reference

In [29]:
%%bash

nanopolish=/home/menkobalazs/nanopolish/nanopolish

fastqdir=/media/storage_3/mb/nanopore_data/analysis/r9.4.1/20200914_1354_6B_PAF27096_e7c9eae6/guppy_v4.0.11_r9.4.1_hac_prom/align_unfiltered/chr22/guppy/pass_cat
bamdir=/media/storage_3/mb/nanopore_data/analysis/r9.4.1/20200914_1354_6B_PAF27096_e7c9eae6/guppy_v4.0.11_r9.4.1_hac_prom/align_unfiltered/chr22/nanopolish/chr22
refdir=/media/storage_3/mb/nanopore_data/analysis/ref_sequences/
dstdir=/media/storage_3/mb/nanopore_data/analysis/r9.4.1/20200914_1354_6B_PAF27096_e7c9eae6/guppy_v4.0.11_r9.4.1_hac_prom/align_unfiltered/chr22/nanopolish/chr22

$nanopolish eventalign \
    --reads $fastqdir/chr22.fastq \
    --bam $bamdir/chr22.sorted.bam \
    --genome $refdir/sequence_chr22.fasta \
    --scale-events > $dstdir/chr22.eventalign.tsv 

In [1]:
import pandas as pd
# Egy read event aligning-ja:
df = pd.read_csv("/media/storage_3/mb/nanopore_data/analysis/r9.4.1/20200914_1354_6B_PAF27096_e7c9eae6/guppy_v4.0.11_r9.4.1_hac_prom/align_unfiltered/chr22/nanopolish/read.eventalign.tsv", sep="\t")
df.head()

,contig,position,reference_kmer,read_index,strand,event_index,event_level_mean,event_stdv,event_length,model_kmer,model_mean,model_stdv,standardized_level
0,gi|568815576|ref|NC_000022.11|,50374804,TCAGGA,0,t,7243,92.91,1.034,0.00075,TCCTGA,94.34,1.64,-0.82
1,gi|568815576|ref|NC_000022.11|,50374805,CAGGAG,0,t,7242,97.23,0.810,0.00225,CTCCTG,98.08,1.92,-0.42
2,gi|568815576|ref|NC_000022.11|,50374806,AGGAGG,0,t,7241,112.16,5.430,0.00075,CCTCCT,115.63,2.47,-1.33
3,gi|568815576|ref|NC_000022.11|,50374807,GGAGGC,0,t,7240,87.52,1.291,0.00150,GCCTCC,89.23,2.16,-0.75
4,gi|568815576|ref|NC_000022.11|,50374808,GAGGCT,0,t,7239,98.30,0.773,0.00150,AGCCTC,99.31,1.86,-0.51
